# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("..")))

## 1. Read Informal Proves
Read the LaTeX file. This file contains the informal proofs of the theorems in the book.

In [2]:
from IPython.display import display, Math, Latex
import re
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof_latex = read_proof("data/proofs/english/problem1/proof1.tex")

# Find the start and end positions
start = proof_latex.find(r"\begin{document}") + len(r"\begin{document}")
end = proof_latex.find(r"\end{document}")

# Extract the content between \begin{document} and \end{document}
informal_proof = proof_latex[start:end].strip()


# Replace any \section{...} with ## ...
informal_proof = re.sub(r"\\section\{([^}]+)\}", r"## \1", informal_proof)

# Display the LaTeX content
display(Latex(informal_proof))

<IPython.core.display.Latex object>

## 2. Extract Triplet proofs from Informal Proofs

In [3]:
from src.phase1.extract_triplets import extract_triplets

# Extract triplets
triplet = extract_triplets(informal_proof)
triplet

Triplet(entities=[Entity(id='theorem_2.3_iv', name='Theorem 2.3(iv)', label='Theorem', type='Theorem'), Entity(id='matrix_A', name='Matrix A', label='Matrix', type='Matrix'), Entity(id='matrix_B', name='Matrix B', label='Matrix', type='Matrix'), Entity(id='entry_AB', name='Entry of AB', label='Entry', type='Matrix Entry'), Entity(id='entry_ATB', name='Entry of (AB)^T', label='Entry', type='Matrix Entry'), Entity(id='entry_BTA', name='Entry of B^T A^T', label='Entry', type='Matrix Entry'), Entity(id='conclusion', name='Conclusion', label='Conclusion', type='Statement')], relations=[Relation(source='theorem_2.3_iv', target='conclusion', type='proves', name='Proves'), Relation(source='matrix_A', target='entry_AB', type='contributes_to', name='Contributes to'), Relation(source='matrix_B', target='entry_AB', type='contributes_to', name='Contributes to'), Relation(source='entry_AB', target='entry_ATB', type='is_equal_to', name='Is equal to'), Relation(source='matrix_B', target='entry_BTA', t

In [4]:
triplet

Triplet(entities=[Entity(id='theorem_2.3_iv', name='Theorem 2.3(iv)', label='Theorem', type='Theorem'), Entity(id='matrix_A', name='Matrix A', label='Matrix', type='Matrix'), Entity(id='matrix_B', name='Matrix B', label='Matrix', type='Matrix'), Entity(id='entry_AB', name='Entry of AB', label='Entry', type='Matrix Entry'), Entity(id='entry_ATB', name='Entry of (AB)^T', label='Entry', type='Matrix Entry'), Entity(id='entry_BTA', name='Entry of B^T A^T', label='Entry', type='Matrix Entry'), Entity(id='conclusion', name='Conclusion', label='Conclusion', type='Statement')], relations=[Relation(source='theorem_2.3_iv', target='conclusion', type='proves', name='Proves'), Relation(source='matrix_A', target='entry_AB', type='contributes_to', name='Contributes to'), Relation(source='matrix_B', target='entry_AB', type='contributes_to', name='Contributes to'), Relation(source='entry_AB', target='entry_ATB', type='is_equal_to', name='Is equal to'), Relation(source='matrix_B', target='entry_BTA', t

## 3. Store Triplets into Neo4J DB

In [5]:
from src.utils.neo4j_utils import Neo4JUtils

# Initialize Neo4JUtils
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))

# Clean the database (delete all nodes and relationships)
neo4j.clean_database()

# Add nodes and relationships with step tracking
for entity in triplet.entities:
    neo4j.create_node(entity)  # Uses the current step (default is 0)
for relation in triplet.relations:
    neo4j.create_relation(relation)  # Uses the current step (default is 0)

# Increment the step counter for the next set of changes
neo4j.increment_step()

# Clean the database (delete nodes and relationships with step > 1)
neo4j.clean_database(step=1)

# Add or modify nodes and relationships in the next step
# Example:
# neo4j.create_node(new_entity)  # This will use the updated step counter (1)
# neo4j.create_relation(new_relation)  # This will use the updated step counter (1)

# Close the connection
neo4j.close()